In [1]:
import requests
import requests_cache
requests_cache.install_cache("cache")
import pandas as pd
from bs4 import BeautifulSoup


import string
import nltk
#from nltk import corpus
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer   # sklearn --- primer machine learning package
from sklearn.neighbors import NearestNeighbors

In [2]:
publisher_id = "254802918413674"

In [3]:
def get_indeed(city, state, keyword, radius, jobtype = "fulltime", js = "json", start = 0, publisher_id = publisher_id):
    url = "http://api.indeed.com/ads/apisearch?"
    if city == "":
        location = state
    else: 
        location = city + ', ' + state
    result = requests.get(url, params = {
            "q": keyword,
            "v": 2,
            "format": js,
            "publisher": publisher_id,
            "l": location,
            "jt": jobtype, # "fulltime", "parttime", "contract", "internship", "temporary"
            "radius": radius,
            "start": start,
            "limit": 100,
            "latlong": 1})
    result = result.json()['results']
    return result

In [23]:
def create_job_list(major):
    '''This function creates the job list of a major and saves the dataframe as a csv file
    Input:  major ----- "statistics", "chemical engineering", "material science"
    Output: shape ----- shape of the dataframe
            job_list -- a dataframe of the job list of the specified major
    '''
    # This is a data frame containing basic information of job posts
    job_list = pd.DataFrame()
    for i in range(0,5000,25):
        temp = pd.DataFrame(get_indeed('','ca', major, 0, start = i))
        job_list = job_list.append(temp, ignore_index = True)
        
    # drop duplicate rows
    job_list = job_list.drop_duplicates(subset = ['jobkey'])
    filename = 'job_ca_' + major + '.csv'
    job_list.to_csv(filename)
    return job_list.shape, job_list

In [24]:
stat_shape, stat_list = create_job_list('statistics')
stat_shape

In [26]:
che_shape, che_list = create_job_list('"chemical engineering"')
che_shape

In [28]:
mse_shape, mse_list = create_job_list('"materials science"')
mse_shape

(347, 20)